In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv('pmsm_temperature_data.csv')
data.head()

,ambient,coolant,u_d,u_q,motor_speed,torque,i_d,i_q,pm,stator_yoke,stator_tooth,stator_winding,profile_id
0,-0.752143,-1.118446,0.327935,-1.297858,-1.222428,-0.250182,1.029572,-0.245860,-2.522071,-1.831422,-2.066143,-2.018033,4
1,-0.771263,-1.117021,0.329665,-1.297686,-1.222429,-0.249133,1.029509,-0.245832,-2.522418,-1.830969,-2.064859,-2.017631,4
2,-0.782892,-1.116681,0.332771,-1.301822,-1.222428,-0.249431,1.029448,-0.245818,-2.522673,-1.830400,-2.064073,-2.017343,4
3,-0.780935,-1.116764,0.333700,-1.301852,-1.222430,-0.248636,1.032845,-0.246955,-2.521639,-1.830333,-2.063137,-2.017632,4
4,-0.774043,-1.116775,0.335206,-1.303118,-1.222429,-0.248701,1.031807,-0.246610,-2.521900,-1.830498,-2.062795,-2.018145,4


In [3]:
data.shape

(998070, 13)

In [4]:
data.columns

Index(['ambient', 'coolant', 'u_d', 'u_q', 'motor_speed', 'torque', 'i_d',
       'i_q', 'pm', 'stator_yoke', 'stator_tooth', 'stator_winding',
       'profile_id'],
      dtype='object')

In [5]:
data['profile_id'].unique().size

52

In [6]:
data['motor_speed'].unique().size

490798

In [7]:
data.describe()

,ambient,coolant,u_d,u_q,motor_speed,torque,i_d,i_q,pm,stator_yoke,stator_tooth,stator_winding,profile_id
count,998070.000000,998070.000000,998070.000000,998070.000000,998070.000000,998070.000000,998070.000000,998070.000000,998070.000000,998070.000000,998070.000000,998070.000000,998070.000000
mean,-0.003905,0.004723,0.004780,-0.005690,-0.006336,-0.003333,0.006043,-0.003194,-0.004396,0.000609,-0.002208,-0.003935,50.732001
std,0.993127,1.002423,0.997878,1.002330,1.001229,0.997907,0.998994,0.997912,0.995686,1.001049,0.999597,0.998343,22.073125
min,-8.573954,-1.429349,-1.655373,-1.861463,-1.371529,-3.345953,-3.245874,-3.341639,-2.631991,-1.834688,-2.066143,-2.019973,4.000000
25%,-0.599385,-1.037925,-0.826359,-0.927390,-0.951892,-0.266917,-0.756296,-0.257269,-0.672308,-0.747265,-0.761951,-0.725622,32.000000
50%,0.266157,-0.177187,0.267542,-0.099818,-0.140246,-0.187246,0.213935,-0.190076,0.094367,-0.057226,0.005085,0.006536,56.000000
75%,0.686675,0.650709,0.358491,0.852625,0.853584,0.547171,1.013975,0.499260,0.680691,0.697344,0.772239,0.725660,68.000000
max,2.967117,2.649032,2.274734,1.793498,2.024164,3.016971,1.060937,2.914185,2.917456,2.449158,2.326668,2.653781,81.000000


In [8]:
data['motor_speed'].value_counts()

-1.222430    4836
-1.222430    4788
-1.222430    4723
-1.222430    4689
-1.222430    4680
-1.222431    4645
-1.222431    4644
-1.222431    4643
-1.222430    4540
-1.222430    4450
-1.222431    4401
-1.222431    4391
-1.222430    4349
-1.222430    4292
-1.222431    4201
-1.222429    4191
-1.222431    4060
-1.222429    3926
-1.222429    3899
-1.222431    3815
-1.222432    3763
-1.222429    3745
-1.222432    3676
-1.222429    3474
-1.222429    3379
-1.222432    3347
-1.222432    3180
-1.222429    3050
-1.222432    2999
-1.222429    2962
             ... 
-0.649571       1
-0.159482       1
-0.521557       1
 1.345073       1
 1.732843       1
 1.380142       1
 1.300079       1
 0.819740       1
-0.413547       1
-0.312841       1
 1.451473       1
 1.528835       1
-0.867959       1
 0.202796       1
 1.912850       1
-0.674868       1
 1.687972       1
-0.424996       1
 1.608853       1
 0.537717       1
 0.843741       1
 0.457863       1
-0.215009       1
 0.305881       1
-0.632973 

In [9]:
data.dtypes

ambient           float64
coolant           float64
u_d               float64
u_q               float64
motor_speed       float64
torque            float64
i_d               float64
i_q               float64
pm                float64
stator_yoke       float64
stator_tooth      float64
stator_winding    float64
profile_id          int64
dtype: object

In [10]:
df2=pd.DataFrame({col: data[col].astype('category').cat.codes for col in data},index=data.index)

In [11]:
df2.dtypes

ambient           int32
coolant           int32
u_d               int32
u_q               int32
motor_speed       int32
torque            int32
i_d               int32
i_q               int32
pm                int32
stator_yoke       int32
stator_tooth      int32
stator_winding    int32
profile_id         int8
dtype: object

In [12]:
df2.head()

,ambient,coolant,u_d,u_q,motor_speed,torque,i_d,i_q,pm,stator_yoke,stator_tooth,stator_winding,profile_id
0,172874,4740,632348,66568,134,254051,660269,249889,1035,33,0,20,0
1,168034,4852,635802,67529,125,255934,660266,250267,1019,38,1,26,0
2,165148,4889,644357,50484,137,255867,660260,250529,1008,50,3,29,0
3,165599,4879,645779,50439,118,256052,660331,245037,1056,52,10,25,0
4,167302,4877,648317,48411,131,256038,660321,245346,1046,46,14,19,0


In [13]:
df2.isnull().sum()

ambient           0
coolant           0
u_d               0
u_q               0
motor_speed       0
torque            0
i_d               0
i_q               0
pm                0
stator_yoke       0
stator_tooth      0
stator_winding    0
profile_id        0
dtype: int64

In [14]:
x=df2.drop(['motor_speed'],axis=1)
y=df2['motor_speed']
x.shape,y.shape

((998070, 12), (998070,))

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=5)

In [16]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((598842, 12), (598842,), (399228, 12), (399228,))

In [17]:
knn=KNeighborsClassifier(n_neighbors=17)
knn.fit(x_train,y_train)

knn.predict([[2,256,123,154,189,754,169,45,856,125,478,123]])

array([476940])

In [18]:
accuracy=knn.score(x_train,y_train)
print('accuracy=',accuracy*100,'%')

accuracy= 7.246151739523948 %
